In [1]:
import json
import time
import os
import glob
import requests

In [2]:
os.chdir("../../")

In [3]:
from src.utils.test_images import image_links
IMAGE_LINKS = image_links

## Step 2:  Load Pre-trained Model from TensorFlow Hub

In [4]:
import tensorflow as tf
import tensorflow_hub as hub

In [5]:
IMAGE_SHAPE = (224, 224)

In [6]:
def load_model():
    mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
    classifier_model = mobilenet_v2

    classifier = tf.keras.Sequential([
        hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE + (3,))
    ])
    print("Model loaded")
    return classifier

## Step 3: Save the loaded Model

In [7]:
def save_model():
    model = load_model()
    ts = int(time.time())
  
    base_file_path = os.getcwd() + "/img-classifier-tfx/src/models/"
    print(base_file_path)

    file_path = base_file_path + str(ts)
    model.save(filepath=file_path, save_format='tf')
    print(f"Model Saved under {file_path}")


In [8]:
from src.utils.test_images import image_links
from src.utils.utilities import is_dir_exists, is_directory_empty

In [9]:
MODEL_LOCATION = os.getcwd() + "/img-classifier-tfx/src/models/"
MODEL_LOCATION

'/Users/ashmi/Scripts/open-source/img-classifier-tfx/src/models/'

In [10]:
if not is_dir_exists(MODEL_LOCATION):
    # Create a new directory because it does not exist
    os.makedirs(MODEL_LOCATION)
if is_directory_empty(MODEL_LOCATION):
    # Load and save the model when pre-loaded model does not exist
    save_model()
else:
    print("Model exists")


Model exists


## Step 4: Serve TF Model using TF Serving

In [11]:
from src.pred.tf_pred import preprocess_image, decode_predictions
from src.app.tf_serve import get_model_name

In [12]:
model_name = get_model_name(MODEL_LOCATION)
# URL at which the model is being served
URL = f'http://localhost:9001/v1/models/{model_name}:predict'
URL

'http://localhost:9001/v1/models/1661517839:predict'

In [13]:

def serve_rest(img_url):
    img = preprocess_image(img_url)
    print("Img preprocessing done!")
    
    data = json.dumps({"signature_name": "serving_default", "instances": img.tolist()})
    headers = {"content-type": "application/json"}
    
    json_response = requests.post(URL, data=data, headers=headers)
    print("json response received")
    
    predictions = json.loads(json_response.text)['predictions']
    
    results = decode_predictions(predictions)
    print("predictions decoded")
    
    return results


## Step 5: Run TF Serving


![alt text](images/run_tf_server/img_10.png "Run TF Serving Step-10")

## Predict Results

In [14]:
if __name__ == "__main__":
    
    try:
        for IMAGE_LINK in IMAGE_LINKS:
            image_url = IMAGE_LINK["url"]
            print(f"Image Link: {image_url}")
            res = serve_rest(image_url)
            print(res)
            print()
    except Exception as e:
        print(e)
        print("Server not running")


Image Link: https://github.com/pytorch/hub/raw/master/images/dog.jpg
Img preprocessing done!
json response received
predictions decoded
{'predicted_label': 'Samoyed', 'probability': 0.6407578587532043}

Image Link: https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg


2022-08-29 17:28:59.951239: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Img preprocessing done!
json response received
predictions decoded
{'predicted_label': 'tiger cat', 'probability': 0.609558641910553}

Image Link: https://media.istockphoto.com/photos/hen-picture-id831791190?k=20&m=831791190&s=170667a&w=0&h=pT2buEaMcxE5Wm4hQz-pXl0Zv-0DdpSMwpDzVcpcKyo=
Img preprocessing done!
json response received
predictions decoded
{'predicted_label': 'hen', 'probability': 0.9590937495231628}

